In [1]:
import xarray as xr

In [2]:
ds = xr.open_dataset('/work/scratch-nopw2/mmuetz/wcrp_hackathon/DYAMOND3_example_data/5km-RAL3/20200120T00.2d.nc')

/home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/pyproj/network.py:59: UserWarning: pyproj unable to set PROJ database path.
  _set_context_ca_bundle_path(ca_bundle_path)
/tmp/ipykernel_939/2883126012.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset('/work/scratch-nopw2/mmuetz/wcrp_hackathon/DYAMOND3_example_data/5km-RAL3/20200120T00.2d.nc')
/tmp/ipykernel_939/2883126012.py:1: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  ds = xr.open_dataset('/work/scratch-nopw2/mmuetz/wcrp_hackathon/DYAMOND3_example_data/5km-RAL3/20200120T00.2d.nc')
/tmp/ipykernel_939/2883126012.py:1: FutureWarning: In a future version of xarray decode_timedelta 

In [3]:
ds

<xarray.Dataset> Size: 23GB
Dimensions:                                                       (time: 13,
                                                                   latitude: 3840,
                                                                   longitude: 5120,
                                                                   time_0: 13,
                                                                   latitude_0: 3841,
                                                                   longitude_0: 5120,
                                                                   time_1: 12,
                                                                   bnds: 2,
                                                                   time_2: 12,
                                                                   time_3: 12)
Coordinates: (12/17)
  * time                                                          (time) datetime64[ns] 104B ...
  * latitude                                                      (latitude) float32 15kB ...
  * longitude                                                     (longitude) float32 20kB ...
    forecast_period                                               (time) timedelta64[ns] 104B ...
    forecast_reference_time                                       datetime64[ns] 8B ...
  * time_0                                                        (time_0) datetime64[ns] 104B ...
    ...                                                            ...
  * time_1                                                        (time_1) datetime64[ns] 96B ...
    forecast_period_1                                             (time_1) timedelta64[ns] 96B ...
  * time_2                                                        (time_2) datetime64[ns] 96B ...
    forecast_period_2                                             (time_2) timedelta64[ns] 96B ...
  * time_3                                                        (time_3) datetime64[ns] 96B ...
    forecast_period_3                                             (time_3) timedelta64[ns] 96B ...
Dimensions without coordinates: bnds
Data variables: (12/27)
    air_pressure_at_sea_level                                     (time, latitude, longitude) float32 1GB ...
    latitude_longitude                                            int32 4B ...
    air_temperature                                               (time_0, latitude, longitude) float32 1GB ...
    atmosphere_cloud_liquid_water_content                         (time, latitude, longitude) float32 1GB ...
    atmosphere_cloud_ice_content                                  (time, latitude, longitude) float32 1GB ...
    m01s30i461                                                    (time, latitude, longitude) float32 1GB ...
    ...                                                            ...
    surface_temperature                                           (time, latitude, longitude) float32 1GB ...
    toa_incoming_shortwave_flux                                   (time_2, latitude, longitude) float32 944MB ...
    toa_outgoing_shortwave_flux                                   (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_shortwave_flux_assuming_clear_sky                (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_longwave_flux                                    (time_3, latitude, longitude) float32 944MB ...
    toa_outgoing_longwave_flux_assuming_clear_sky                 (time_3, latitude, longitude) float32 944MB ...
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   13.5
    Conventions:  CF-1.7

In [4]:
ds = ds[['air_pressure_at_sea_level', 'surface_temperature']]
ds = ds.drop_vars(['forecast_period', 'forecast_reference_time', 'height', 'height_0'])

In [5]:
print(ds)

<xarray.Dataset> Size: 2GB
Dimensions:                    (time: 13, latitude: 3840, longitude: 5120)
Coordinates:
  * time                       (time) datetime64[ns] 104B 2020-01-20 ... 2020...
  * latitude                   (latitude) float32 15kB -89.98 -89.93 ... 89.98
  * longitude                  (longitude) float32 20kB 0.03516 0.1055 ... 360.0
Data variables:
    air_pressure_at_sea_level  (time, latitude, longitude) float32 1GB ...
    surface_temperature        (time, latitude, longitude) float32 1GB ...
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   13.5
    Conventions:  CF-1.7


In [10]:
import easygems.remap as egr

weights_path = '/gws/nopw/j04/hrcm/mmuetz/weights/regrid_weights_N2560_hpz10.cyclic_lon.lon=(0.03515625,359.96484375,5120).lat=(-89.9765625,89.9765625,3840).nc'
weights = xr.load_dataset(weights_path)
npix = 12 * 4**10

xr.apply_ufunc(
    egr.apply_weights, 
    ds,
    kwargs=weights,
    keep_attrs=True,
    input_core_dims=[["latitude", 'longitude']],
    output_core_dims=[["cell"]],
    output_dtypes=["f4"],
    vectorize=True,
    dask="parallelized",
    dask_gufunc_kwargs={
        "output_sizes": {"cell": npix},
    }   
)

IndexError: index 2459520 is out of bounds for axis 0 with size 3840

In [12]:
def wrapper(x, y):
    print(f"received x of type {type(x)}, shape {x.shape}")
    print(f"received y of type {type(y)}")
    

> /home/users/mmuetz/miniforge3/envs/hackathon_env/lib/python3.12/site-packages/easygems/remap.py(60)apply_weights()
     56 
     57     See also:
     58         `compute_weights`
     59     """
---> 60     return np.where(valid, (var[src_idx] * weights).sum(axis=-1), np.nan)

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


ipdb>  q


In [17]:
import healpix as hp

In [19]:
order = zoom = 10
nside = hp.order2nside(order)
npix = hp.nside2npix(nside)

In [21]:
npix == 12 * 4**10

True